In [29]:
from chat_GPT import Chat
import pandas as pd
from tqdm.auto import tqdm

Importing the test data

In [42]:
data = pd.read_csv("../data/test/clean_test.csv")
data.head(10)

,id,comment_text,label
0,0001ea8717f6de06,Thank you for understanding I think very highl...,0
1,000247e83dcc1211,Dear god this site is horrible,0
2,0002f87b16116a7f,Somebody will invariably try to add Religion ...,0
3,0003e1cccfd5a40a,It says it right there that it IS a type T...,0
4,00059ace3e3e9a53,Before adding a new product to the list m...,0
5,000663aff0fffc80,this other one from,0
6,000689dd34e20979,Reason for banning throwing This article ...,0
7,000844b52dee5f3f,blocked from editing Wikipedia,0
8,00091c35fa9d0465,Arabs are committing genocide in Iraq but no ...,1
9,000968ce11f5ee34,Please stop If you continue to vandalize Wikip...,0


Creating a subset of the data to test the GPT-3 model

In [27]:
messages = data['comment_text'][0:1000].to_list()

## Classify the messages using GPT-3 turbo

Importing the chat_GPT class and creating a chat object to classify the messages. We used chunks of 10 messages to reduce the number tokens per request to the API.

In [30]:
responses = []
chat = Chat()

for i in tqdm(range(0, len(messages), 10)):
    chunk = messages[i:i+10] 
    responses += chat.batch_create_chats(chunk)

  0%|          | 0/100 [00:00<?, ?it/s]

BadRequestError: Error code: 400 - {'error': {'message': "Sorry! We've encountered an issue with repetitive patterns in your prompt. Please try again with a different prompt.", 'type': 'invalid_request_error', 'param': 'prompt', 'code': 'invalid_prompt'}}

The process has been interrupted by the api after 800 messages.

Now save the responses to a local csv file.

In [ ]:
data = data[0:800]
data.loc[:, 'gpt_label'] = responses
data['gpt_label'] = data['gpt_label'].astype(int)
data.to_csv("../data/test/800_test_labeled_gpt.csv", index=False)

Calculate the F1 Score for the GPT-3 labels

In [39]:
from sklearn.metrics import f1_score

f1_score(data['label'], data['gpt_label'], average='weighted')

0.8215979843308009

The F1Score is 0.82 which is a relatively good score. Now we'll try to outperform this score using a our transformer model.